# Let's build an agentic search loop!

This notebook has a basic agentic search loop

* We have a set of furniture in our catalog
* We tell the Agent our preferences
* The agent uses the search tool to recommend furniture

In this notebook we mostly get a feel for how the overall loop works by unrolling it step by step

In [ ]:
!pip install git+https://github.com/softwaredoug/cheat-at-search.git
from cheat_at_search.data_dir import mount
mount(use_gdrive=True)    # colab, share data across notebook runs on gdrive
# mount(use_gdrive=False) # <- colab without gdrive
# mount(use_gdrive=False, manual_path="/path/to/directory")  # <- force data path to specific directory, ie you're running locally.


  Cloning https://github.com/softwaredoug/cheat-at-search.git to /tmp/pip-req-build-jzv9npeu
  Running command git clone --filter=blob:none --quiet https://github.com/softwaredoug/cheat-at-search.git /tmp/pip-req-build-jzv9npeu
  Resolved https://github.com/softwaredoug/cheat-at-search.git to commit 6a08d097f1d6eaa068fb61af47c621df1682f5e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.3/745.3 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 21.6 MB/s eta 0:00:00
  Created wheel for cheat_at_search: filename=cheat_at_search-0.1.0-py3-none-any.whl size=64150 sha256=bc2e8078e2fc527fcee519d231d6892564a6d1a5550dd365aeeeb97aaeeb0211
  Stored in directory: /tmp/pip-ephem-wheel-cache-9_8kjde1/wheels/ea/96/d3/e69453e5f86e6f891864e5b5baf2b11ffda2d0981

## Get an OpenAI Key

This will prompt you for an OpenAI Key to interact with GPT-5

In [ ]:
from cheat_at_search.data_dir import key_for_provider
from openai import OpenAI

OPENAI_KEY = key_for_provider("openai")

openai = OpenAI(api_key=OPENAI_KEY)

## Load the Wayfair corpus

We'll recommend products only from this corpus

In [ ]:
from cheat_at_search.wands_data import corpus

corpus['category'] = corpus['category'].str.strip()

corpus

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,doc_id,title,description,category,sub_category,cat_subcat
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0,"[overallwidth-sidetoside:64.7, dsprimaryproduc...",0,solid wood platform bed,"good , deep sleep can be quite difficult to ha...",Furniture,Bedroom Furniture,Furniture / Bedroom Furniture
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0,"[capacityquarts:7, producttype : slow cooker, ...",1,all-clad 7 qt . slow cooker,"create delicious slow-cooked meals , from tend...",Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0,"[features : keep warm setting, capacityquarts:...",2,all-clad electrics 6.5 qt . slow cooker,prepare home-cooked meals on any schedule with...,Kitchen & Tabletop,Small Kitchen Appliances,Kitchen & Tabletop / Small Kitchen Appliances
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0,"[overallwidth-sidetoside:3.5, warrantylength :...",3,all-clad all professional tools pizza cutter,this original stainless tool was designed to c...,Browse By Brand,All-Clad,Browse By Brand / All-Clad
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0,"[compatibledoorthickness:1.375 '' , countryofo...",4,baldwin prestige alcott passage knob with roun...,the hardware has a rich heritage of delivering...,Home Improvement,Doors & Door Hardware,Home Improvement / Doors & Door Hardware
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42989,42989,malibu pressure balanced diverter fixed shower...,Shower Panels,Home Improvement / Bathroom Remodel & Bathroom...,the malibu pressure balanced diverter fixed sh...,producttype : shower panel|spraypattern : rain...,3.0,4.5,2.0,"[producttype : shower panel, spraypattern : ra...",42989,malibu pressure balanced diverter fixed shower...,the malibu pressure balanced diverter fixed sh...,Home Improvement,Bathroom Remodel & Bathroom Fixtures,Home Improvement / Bathroom Remodel & Bathro...
42990,42990,emmeline 5 piece breakfast dining set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,,basematerialdetails : steel| : gray wood|ofhar...,1314.0,4.5,864.0,"[basematerialdetails : steel, : gray wood, of...",42990,emmeline 5 piece breakfast dining set,,Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42991,42991,maloney 3 piece pub table set,Dining Table Sets,Furniture / Kitchen & Dining Furniture / Dinin...,this pub table set includes 1 counter height t...,additionaltoolsrequirednotincluded : power dri...,49.0,4.0,41.0,[additionaltoolsrequirednotincluded : power dr...,42991,maloney 3 piece pub table set,this pub table set includes 1 counter height t...,Furniture,Kitchen & Dining Furniture,Furniture / Kitchen & Dining Furniture
42992,42992,fletcher 27.5 '' wide polyester armchair,Teen Lounge Furniture|Accent Chairs,Furniture / Living Room Furniture / Chairs & S...,"bring iconic , modern style to your space in a...",legmaterialdetails : rubberwood|backheight-sea...,1746.0,4.5,1226.0,"[legmaterial

### Index the furniture

We'll index title and description with basic stemming to be able to retrieve them

In [ ]:
from searcharray import SearchArray
from cheat_at_search.tokenizers import snowball_tokenizer

corpus['title_snowball'] = SearchArray.index(corpus['title'].fillna(''), snowball_tokenizer)
corpus['description_snowball'] = SearchArray.index(corpus['description'].fillna(''), snowball_tokenizer)

2026-02-06 18:04:52,962 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2026-02-06 18:04:52,970 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2026-02-06 18:04:52,972 - searcharray.indexing - INFO - Tokenizing 42994 documents


...

INFO:searcharray.indexing:Batch tokenization complete


2026-02-06 18:05:02,123 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2026-02-06 18:05:02,428 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


## Create a furniture products search function

Here is a function that searches a Wayfair product dataset. It's just a Python function that returns top 10 pieces of furniture.

Right now we'll call it directly, soon we'll help ChatGPT interact with this.

In [ ]:
import numpy as np
from typing import Union

from pydantic import BaseModel, Field
from typing import Optional, Literal


Categories = Literal['Furniture',
                     'Home Improvement',
                     'Décor & Pillows',
                     'Outdoor',
                     'Storage & Organization',
                     'Lighting',
                     'Rugs',
                     'Bed & Bath',
                     'Kitchen & Tabletop',
                     'Baby & Kids',
                     'School Furniture and Supplies',
                     'Appliances',
                     'Holiday Décor',
                     'Commercial Business Furniture',
                     'Pet',
                     'Contractor',
                     'Sale',
                     'Foodservice ',
                     'Reception Area',
                     'Clips']

def search_furniture(keywords: str,
                     categories: Optional[list[Categories]] = None
                     ) -> list[dict[str, Union[str, int, float]]]:
    """Search the available furniture products, get top 10 furniture.

    This is just a naive BM25 / keyword search of the product title and description.
    Don't expect sophisticated synonyms or semantic search. Just basic keyword with
    some stemming.

    Optionally you can filter to a list of categories. We will retrieve any item that
    matches any of the categories. No filtering occurs with an empty list.

    """
    print("search", keywords)
    required_keywords = [term[1:] for term in keywords.split() if term.startswith("+")]
    bm25_scores = np.zeros(len(corpus))
    for term in snowball_tokenizer(keywords):
        bm25_scores += corpus['title_snowball'].array.score(term) * 7
        bm25_scores += corpus['description_snowball'].array.score(term) * 4

    for required_term in snowball_tokenizer(" ".join(required_keywords)):
        required_score = (corpus['title_snowball'].array.score(required_term) +
                          corpus['description_snowball'].array.score(required_term))
        bm25_scores[required_score == 0] = 0

    if categories:
        bm25_scores[~corpus['category'].isin(categories)] = 0

    top_k_indices = np.argsort(bm25_scores)[-10:][::-1]
    bm25_scores = bm25_scores[top_k_indices]
    top_movies = corpus.iloc[top_k_indices].copy()
    top_movies.loc[:, 'score'] = bm25_scores

    results = []
    for id, row in top_movies.iterrows():
        results.append({
            'id': row['doc_id'],
            'title': row['title'],
            'description': row['description'],
            'category': row['category'],
            'score': row['score']
        })
    return results



search_furniture("geometric style +couch",
                 categories=['Furniture'])

search geometric style +couch


[{'id': 824,
  'title': 'double chaise lounge floor couch',
  'description': "the multi-functional lazy sofa is becoming a popular trend for people to enjoy themselves after their tiring work every day . it is great for almost every place , such as living room , bedroom , home office , dorm room , balcony , and outdoor space , and suitable for watching tv , play games , working on a laptop , or take a nap in it . you wo n't be disappointed with this purchase . lean back and get comfortable after a long day with the leisure sofa bed with a built-in 5 gear back adjuster system and take floor comfort to the next level . the built-in back adjuster system allows the chair to easily take on 5 different angled positions from 90 degrees in an upright chair position to 180 degrees flatbed position , satisfying any and all posture requirements for a customized seating experience .",
  'category': 'Furniture',
  'score': 25.523414611816406},
 {'id': 23758,
  'title': 'double chaise lounge sofa fl

## Describe the search tool to the LLM

There is a specific schema for telling OpenAI about our tools / functions. However, the cheat at search library has added some conveniences:

* We use the function name as the name to OpenAI
* We use the doc string to get a description
* The typing information gets encoded in parameters and return value

So IMPORTANTLY -- all these things are part of the prompt

### Annoying serialization / deserialization

When we get it in an OpenAI-friendly format, we also keep around some book-keeping for annoying serialization / deserialization of the arguments

With this we get some plumbing information in a 3-tuple
* The arguments to pass (as one pydantic struct)
* The tool as OpenAI sees it
* The function to call to delegate to this tool

Don't get too lost in the weeds here. In future notebooks, cheat-at-search helper code will just do this for you behind the scenes.

In [ ]:
from cheat_at_search.agent.pydantize import make_tool_adapter

search_tool = make_tool_adapter(search_furniture)

tool_info = {search_furniture.__name__: search_tool}
tool_info

{'search_furniture': (cheat_at_search.agent.pydantize.Search_furnitureArgs,
  {'type': 'function',
   'name': 'search_furniture',
   'description': "Search the available furniture products, get top 10 furniture.\n\n    This is just a naive BM25 / keyword search of the product title and description.\n    Don't expect sophisticated synonyms or semantic search. Just basic keyword with\n    some stemming.\n\n    Optionally you can filter to a list of categories. We will retrieve any item that\n    matches any of the categories. No filtering occurs with an empty list.",
   'parameters': {'properties': {'keywords': {'title': 'Keywords',
      'type': 'string'},
     'categories': {'anyOf': [{'items': {'enum': ['Furniture',
          'Home Improvement',
          'Décor & Pillows',
          'Outdoor',
          'Storage & Organization',
          'Lighting',
          'Rugs',
          'Bed & Bath',
          'Kitchen & Tabletop',
          'Baby & Kids',
          'School Furniture and Supp

In [ ]:
import json
print(json.dumps(tool_info['search_furniture'][1], indent=2))

{
  "type": "function",
  "name": "search_furniture",
  "description": "Search the available furniture products, get top 10 furniture.\n\n    This is just a naive BM25 / keyword search of the product title and description.\n    Don't expect sophisticated synonyms or semantic search. Just basic keyword with\n    some stemming.\n\n    Optionally you can filter to a list of categories. We will retrieve any item that\n    matches any of the categories. No filtering occurs with an empty list.",
  "parameters": {
    "properties": {
      "keywords": {
        "title": "Keywords",
        "type": "string"
      },
      "categories": {
        "anyOf": [
          {
            "items": {
              "enum": [
                "Furniture",
                "Home Improvement",
                "D\u00e9cor & Pillows",
                "Outdoor",
                "Storage & Organization",
                "Lighting",
                "Rugs",
                "Bed & Bath",
                "Kitchen & T

## Gather initial prompts

* System prompt - the general task, to lookup furniture in our catalog to recommend
* User prompt - what the user has given as a task (here listing the movies they like)



In [ ]:
system_prompt = """
Users are coming to explore a catalog of furniture.

Use the search tool (search_furniture) to help them
"""

inputs = []
inputs.append({"role": "system", "content": system_prompt})

prompt = """
Help me find a modern couch with geometric style
"""

inputs.append({"role": "user", "content": prompt})
inputs

[{'role': 'system',
  'content': '\nUsers are coming to explore a catalog of furniture.\n\nUse the search tool (search_furniture) to help them\n'},
 {'role': 'user',
  'content': '\nHelp me find a modern couch with geometric style\n'}]

## Make a single call to the LLM (non-tool)

We make a call to the agent, and get some decent recommendations back. But without using the catalog to lookup what's available.

In [ ]:
resp = openai.responses.create(
    model="gpt-5",
    input=inputs,
)
resp

Response(id='resp_08039186f133f4b50069862d4f81f081939e9dbb831bfac659', created_at=1770401103.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_08039186f133f4b50069862d5013348193b74a8dde25c1b5ec', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseReasoningItem(id='rs_08039186f133f4b50069862d5908448193bdd20c908a6cb98a', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseReasoningItem(id='rs_08039186f133f4b50069862d5fb56481938313581d9617efd8', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_08039186f133f4b50069862d5ffd608193beefb6bb9b851183', content=[ResponseOutputText(annotations=[], text='{"query":"modern geometric couch","filters":{"category":"Sofas & Couches","style":["Modern","Contemporary"]},"keywords":["geometric","angular","clean lines"]

## Tell the agent about a tool

We tell the agent about the tools it can use. Note the agent doesn't directly "call" the tools (there's no magic backdoor from OpenAI -> this notebook). Insetad, once the agent knows about the tool it can REQUEST we call them.

In [ ]:
resp = openai.responses.create(
    model="gpt-5",
    input=inputs,
    tools=[tool[1] for tool in tool_info.values()],
)
inputs += resp.output
resp

Response(id='resp_0c405b4cd5869f3e0069862d638ec08197a9c550adc6532eac', created_at=1770401123.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_0c405b4cd5869f3e0069862d6520808197b2bfb3bf509038c2', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"keywords":"modern geometric couch sofa","categories":["Furniture"]}', call_id='call_3YvAmSiCjufIHweqrIUFOqbz', name='search_furniture', type='function_call', id='fc_0c405b4cd5869f3e0069862d6964548197a3ece2be1cc0464b', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='search_furniture', parameters={'properties': {'keywords': {'title': 'Keywords', 'type': 'string'}, 'categories': {'anyOf': [{'items': {'enum': ['Furniture', 'Home Improvement', 'Décor & Pillows', 'Outdoor', 'Storage & Organization', 'Lighting', 'Rugs'

In [ ]:
resp.output

[ResponseReasoningItem(id='rs_0c405b4cd5869f3e0069862d6520808197b2bfb3bf509038c2', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFunctionToolCall(arguments='{"keywords":"modern geometric couch sofa","categories":["Furniture"]}', call_id='call_3YvAmSiCjufIHweqrIUFOqbz', name='search_furniture', type='function_call', id='fc_0c405b4cd5869f3e0069862d6964548197a3ece2be1cc0464b', status='completed')]

## Issue the requested calls to your search tool

Now we do the magic of calling the tools directly

You can ignore the `annoying_tool_marshalling` its doing some lookups and plumbings to go between the JSON arguments and the Python world we have here.

The important thing is that we recieve a tool call request, we call the requested tool (here by doing a lookup and getting `tool_fn` that just wraps the search function)

Then we go on to append those all back into th inputs, with a JSON response, the call id, and a note to OpenAI this is a "function_call_output"

In [ ]:
def call_tool(item) -> dict:

    # Lookup how the agent wants to call the tool
    tool_name = item.name
    tool = tool_info[tool_name]
    ToolArgsModel = tool[0]
    tool_fn = tool[2]
    fn_args: ToolArgsModel = ToolArgsModel.model_validate_json(item.arguments)

    # The tool call function itself (ie search)
    # wrapped in something helping with serialization
    py_resp, json_resp = tool_fn(fn_args)

    # 4. Provide function call results to the model
    return {
        "type": "function_call_output",
        "call_id": item.call_id,
        "output": json_resp,
    }


for item in resp.output:
    if item.type == "function_call":
        tool_name = item.name

        # *** Get the tool, and package
        # up the call to the tool (our python function)
        tool_response = call_tool(item)
        # 4. Provide function call results to the model
        inputs.append(tool_response)



search modern geometric couch sofa


## Call LLM again with tool responses

Now we tell the LLM about the tool responses.

It may ask to search more (ie we should continue the loop). Or it may be done and have a final response


In [ ]:
resp = openai.responses.create(
    model="gpt-5",
    input=inputs,
    tools=[tool[1] for tool in tool_info.values()],
)
inputs += resp.output

resp

Response(id='resp_0c405b4cd5869f3e0069862d69f1548197bb054ad1f15487fe', created_at=1770401129.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_0c405b4cd5869f3e0069862d6a620081979d26cbcf015d5847', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"keywords":"geometric pattern sofa couch","categories":["Furniture"]}', call_id='call_8hrH161yYaJLij6aySHlXCGx', name='search_furniture', type='function_call', id='fc_0c405b4cd5869f3e0069862d6dd91c81978650020b69c2942d', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='search_furniture', parameters={'properties': {'keywords': {'title': 'Keywords', 'type': 'string'}, 'categories': {'anyOf': [{'items': {'enum': ['Furniture', 'Home Improvement', 'Décor & Pillows', 'Outdoor', 'Storage & Organization', 'Lighting', 'Rugs

## Put it all in one loop

In [ ]:
import textwrap

system_prompt = """
Users are coming to explore a catalog of furniture.

Use the search tool (search_furniture) to help them.

Use trial and error to figure out how best use the search tool.
"""

def agentic_search(query: str, summary=True) -> str:

    inputs = []
    inputs.append({"role": "system", "content": system_prompt})

    inputs.append({"role": "user", "content": query})


    tool_calls = True
    resp = None
    while tool_calls:
        resp = openai.responses.create(
            model="gpt-5",
            input=inputs,
            tools=[tool[1] for tool in tool_info.values()],
            reasoning={
                "effort": "auto",
                "summary": "auto" if summary else "none"
            },
            text_format=SearchResults
        )
        inputs += resp.output
        if summary:
            for item in resp.output:
                if item.type == "reasoning":
                    print("Reasoning:")
                    for summary_item in item.summary:
                        print(textwrap.fill(summary_item.text, 80), "\n")
                    item.summary = []

        for item in resp.output:
            tool_calls = False
            if item.type == "function_call":
                tool_calls = True
                # *** Get the tool, and package
                # up the call to the tool (our python function)
                tool_response = call_tool(item)

                # 4. Provide function call results to the model
                inputs.append(tool_response)
    return resp

resp = agentic_search("I want a geometric sofa with square pillows")

NameError: name 'SearchResults' is not defined

In [ ]:
print(resp.output[-1].content[-1].text)